## Setup

Install the required packages from `requirements.txt`

In [1]:
# Enable automatic reloading of modules
%reload_ext autoreload
%autoreload 2

In [2]:
import os
import openai
from dotenv import load_dotenv

# Load environment variables from a .env file
load_dotenv()

True

In [3]:
# Set the OpenAI API key from the environment variable
openai.api_key = os.getenv("OPENAI_API_KEY")

In [4]:
from llama_index.core.node_parser import SentenceWindowNodeParser
from llama_index.core.node_parser import SentenceSplitter

# create the sentence window node parser w/ default settings
node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=5,
    window_metadata_key="window",
    original_text_metadata_key="original_text",
)

# base node parser is a sentence splitter
text_splitter = SentenceSplitter()

c:\Users\saad\OneDrive\Bureau\LlamaChat-RAG-S2B\.conda\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


We'll define two crucial models :

LLM (Language Model): The OpenAI model with the configuration "gpt-3.5-turbo" and a temperature setting of 0.1. This model is responsible for generating responses based on user queries and relevant context.

Embedding Model: The HuggingFaceEmbedding model with the model name "sentence-transformers/all-mpnet-base-v2" and a maximum sequence length of 512. This model is used to create vector embeddings for individual text chunks.

In [5]:
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.huggingface import HuggingFaceEmbedding


# Select Embedding Model and LLM
llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)

embed_model = HuggingFaceEmbedding(
    model_name="sentence-transformers/all-mpnet-base-v2", max_length=512
)

### Settings

In [6]:
from llama_index.core import Settings

# Assign the LLM, embedding model, and text splitter to Settings
Settings.llm = llm
Settings.embed_model = embed_model
Settings.text_splitter = text_splitter

## Lmm Monitoring - Callbacks Integration
#### Using Langfuse

In [7]:
# Set the OpenAI API key from the environment variable
langfuse_public_key = os.getenv("LANGFUSE_PUBLIC_KEY")
langfuse_secret_key = os.getenv("LANGFUSE_SECRET_KEY")

In [8]:
from llama_index.core.callbacks import CallbackManager
from langfuse.llama_index import LlamaIndexCallbackHandler
#  
langfuse_callback_handler = LlamaIndexCallbackHandler(
    public_key=langfuse_public_key,
    secret_key=langfuse_secret_key,
    host="https://cloud.langfuse.com"
)
Settings.callback_manager = CallbackManager([langfuse_callback_handler])

## Prompt Helper

In [9]:
from llama_index.core import Settings

# Define the prompt helper as a dictionary
prompt_helper = {
    "context_window": 4900,
    "num_output": 256,
    "chunk_overlap_ratio": 0.1,
    "chunk_size_limit": 1000
}

# Assign the prompt helper dictionary to the Settings
Settings.prompt_helper = prompt_helper

## Data Loading, Index Building
In this section, we load the data and construct the vector index.

In [10]:
from llama_index.core import SimpleDirectoryReader

# List of PDF files to be read
input_files = [
    "./data/Grasslan_carbon_sequ.pdf",
    "./data/Rot_gas_serre.pdf"

]

# Create a SimpleDirectoryReader instance
documents = SimpleDirectoryReader(input_files=input_files).load_data()

## Extract Nodes

We identify and extract the set of nodes that will be stored in the VectorIndex. This set encompasses nodes processed by the sentence window parser and the "base" nodes extracted using the standard parser.

In [11]:
nodes = node_parser.get_nodes_from_documents(documents)

In [12]:
base_nodes = text_splitter.get_nodes_from_documents(documents)

for idx, node in enumerate(base_nodes):
    node.id_ = f"doc_{idx}"
    print(node)

Node ID: doc_0
Text: Animal (2010), 4:3, pp 334–350 &The Animal Consortium 2009
doi:10.1017/S1751731109990784animal Mitigating the greenhouse gas
balance of ruminant production systems through carbon sequestration in
grasslands J. F . Soussana1-, T. Tallec1and V. Blanfort1,2 1INRA
UR0874, UREP Grassland Ecosystem Research, 234, Avenue du Bre ´zet,
Clermont-Ferrand, ...
Node ID: doc_1
Text: This will require avoidingland use changes that reduce ecosystem
soil C stocks (e.g.deforestation, ploughing up long-term grasslands)
and a cautious management of pastures, aiming at preserving
andrestoring soils and their soil organic matter content. Combinedwith
other mitigation measures, such as a reduction in the useof N
fertilisers, of foss...
Node ID: doc_2
Text: and provide a variety of goods and services to support ﬂora,
fauna, and human populations worldwide. On aglobal scale, livestock
use 3.4 billion hectares of grazingland (i.e. grasslands and
rangelands), in addition to animal feed produ

## Build Index

We build both the sentence index, as well as the “base” index.

In [13]:
from llama_index.core import VectorStoreIndex

# Create a VectorStoreIndex instance using the 'nodes' dataset
sentence_index = VectorStoreIndex(nodes)

In [14]:
# Create another VectorStoreIndex instance using the 'base_nodes' dataset
base_index = VectorStoreIndex(base_nodes)

## Querying

#### Querying with window method

In [15]:
from llama_index.core.postprocessor import MetadataReplacementPostProcessor

# Create a QueryEngine instance using the window retrieval
query_engine = sentence_index.as_query_engine(
    similarity_top_k=5,
    # the target key defaults to `window` to match the node_parser's default
    node_postprocessors=[
        MetadataReplacementPostProcessor(target_metadata_key="window")
    ],
)

#### Querying with normal VectorStoreIndex

In [16]:
# Create a QueryEngine instance using base retrieval
base_query_engine = base_index.as_query_engine(similarity_top_k=2)

If that didn't work, we should bump up the top k! (example k = 4) This will be slower and use more tokens compared to the sentence window index.

## Use Gradio UI to interact with our chatbot

In [73]:
import gradio as gr
import time


with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.ClearButton([msg, chatbot])

    def respond(message, chat_history):
        
        # bot_message = str(base_query_engine.query(message))
        bot_message = str(base_query_engine.query(message))
        chat_history.append((message, bot_message))
        
        time.sleep(0.05)
        return "", chat_history

    msg.submit(respond, [msg, chatbot], [msg, chatbot])

demo.launch()

Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


# Analysis

We examine both the initial sentence retrieved for each node and the specific set of sentences sent to the LLM.

In [71]:
window_response = query_engine.query("What does NCS mean?")

sentence = window_response.source_nodes[0].node.metadata["original_text"]
window = window_response.source_nodes[0].node.metadata["window"]

print(f"Original Sentence: {sentence}")
print("------------------")
print(f"Window: {window}")

Original Sentence: A positive NCS denotes a net carbon accumulation in grassland ecosystems. 
------------------
Window: Table 1 Literature survey of net C storage (NCS) at grassland sites using different methods: C ﬂux balance (A), grassland soil C inventory (B), soil C change after a change in grassland management
(C), and farm scale ﬂux measurements (D).  A positive F CO2represents a net C uptake from the ecosystem.  A positive NCS denotes a net carbon accumulation in grassland ecosystems.  All ﬂuxes are in g
C/m2per year
Grassland type and MAT MAP F CO2 Fharvest Fmanure Duration
management Location ( 8C) (mm) (g C/m2per year) NCS (months) Method References Notes
A. Flux balance
Alpine extensive pasture
and hay meadowMount Rigi, Central
Switerland8.4 991 2172 183 0 2355 12 Eddy covariance Rogiers et al .  (2008) Drained organic soil
Grazed peat-pasture Waikato, New Zealand 15 1281 24.5 619 n.d.  2106 12 Eddy covariance Nieveen et al .  (2005) Drained peat soil
Extensive grazed pastu

In [19]:
for source_node in window_response.source_nodes:
    print("------------------")
    print(source_node.node.metadata["original_text"])

------------------
Advanced (Tier 3) and veriﬁable methodolo-
gies still need to be developed in order to include GHGremovals obtained by farm scale mitigation options inagriculture, forestry and land use (AFOLU sector, IPCC,2006) national GHG inventories.

------------------
The GHG inventory methodology usedby IPCC (IPCC, 1996 and 2006) only includes, however,farm emissions in the agriculture sector. 
------------------
Nevertheless, sites that were intensively managed bygrazing and cutting had a negative NGHG and were
therefore estimated to be GHG sources in CO
2equivalents.

------------------
The mean on-site NGHG reached 198 g CO 2equiva-
lents/m2per year, indicating a sink for the atmosphere.

------------------
The net emissions of GHGs (methane, nitrousoxide and carbon dioxide) are related to C and nitrogen
ﬂows and to environmental conditions.



Here, we observe that the sentence window index effortlessly identified two nodes discussing SOC.

**N.B:** although the embeddings are based solely on the original sentence, the LLM also comprehends the context of SOC during its reading process.

In [20]:
vector_response = base_query_engine.query("What is GHG?")

for node in vector_response.source_nodes:
    print("GHG mentioned?", "GHG" in node.node.text)
    print("--------")

GHG mentioned? True
--------
GHG mentioned? True
--------


In [21]:
print(vector_response.source_nodes[0].node.text)

The attributed GHG
balance was positive for grazed sites (indicating a sink
activity), but was negative for cut and mixed sites (indi-cating a source activity) (Table 2). Therefore, a grazingmanagement seems to be a better strategy for removingGHG from the atmosphere than a cutting management.However, given that the studied sites differed in manyrespects (climate, soil and vegetation) (Soussana
et al .,
2007), this hypothesis needs to be further tested.
Taken together, these results show that managed grass-
lands have a potential to remove GHG from the atmo-
sphere, but that the utilisation of the cut herbage byruminants may lead to large non-CO
2GHG emissions in
farm buildings, which may compensate this sink activity.Data from a larger number of ﬂux sites and from long-termexperiments will be required to upscale these results atregional scale and calculate GHG balance for a range of
production systems. In order to further reduce uncertainties,
C and N ﬂuxes are investigated for a numb

# Evaluation

We'll do an evaluation to assess the effectiveness of the sentence window retriever in comparison to the base retriever.

We establish or load an evaluation benchmark dataset and proceed to execute various assessments on it.

In [22]:
from llama_index.core.evaluation import DatasetGenerator

from llama_index.llms.openai.base import OpenAI
import nest_asyncio
import random

nest_asyncio.apply()

In [57]:
len(base_nodes)

61

In [43]:
num_nodes_eval = 10

# there are 61 nodes total. Take the first 25 to generate question
sample_eval_nodes = random.sample(base_nodes[:25], num_nodes_eval)

# generate questions from the largest chunks (1024)
dataset_generator = DatasetGenerator(
    sample_eval_nodes,
    llm=OpenAI(model="gpt-3.5-turbo"),
    show_progress=True,
    num_questions_per_chunk=2,
)

C:\Users\saad\AppData\Local\Temp\ipykernel_10936\4187532567.py:7: DeprecationWarning: Call to deprecated class DatasetGenerator. (Deprecated in favor of `RagDatasetGenerator` which should be used instead.)
  dataset_generator = DatasetGenerator(


In [ ]:
eval_dataset = await dataset_generator.agenerate_dataset_from_nodes()

In [45]:
eval_dataset.save_json("data/ipcc_eval_qr_dataset.json")

### Compare Results

In [46]:
import nest_asyncio

nest_asyncio.apply()

In [47]:
max_samples = 10

eval_qs = eval_dataset.questions
ref_response_strs = [r for (_, r) in eval_dataset.qr_pairs]

In [ ]:
import asyncio

# Define a function to get responses
async def aget_responses(questions, query_engine, show_progress=True):
    tasks = []
    for question in questions:
        tasks.append(query_engine.aquery(question))
    return await asyncio.gather(*tasks)

# Get responses using the defined function
base_pred_responses = await aget_responses(eval_qs[:max_samples], base_query_engine, show_progress=True)
pred_responses = await aget_responses(eval_qs[:max_samples], query_engine, show_progress=True)

# Convert responses to strings
pred_response_strs = [str(p) for p in pred_responses]
base_pred_response_strs = [str(p) for p in base_pred_responses]

##### The first evaluation question.

In [49]:
eval_qs[0]

'How does soil carbon sequestration contribute to mitigating greenhouse gas emissions in ruminant production systems, and what are some management practices that can enhance carbon sequestration in grasslands?'

##### The first reference answer.

In [50]:
ref_response_strs[0]

'Soil carbon sequestration plays a significant role in mitigating greenhouse gas emissions in ruminant production systems by acting as an enhanced sink for carbon. This process can be directly measured by changes in soil organic carbon stocks and indirectly by measuring the net balance of carbon fluxes. Grasslands have been shown to sequester an average of 5,630 g C/m2 per year through soil organic carbon stocks and 2231 and 77 g C/m2 per year for drained organic and mineral soils, respectively, through carbon flux balance.\n\nTo enhance carbon sequestration in grasslands, various management practices can be implemented, including:\n1. Avoiding soil tillage and the conversion of grasslands to arable use.\n2. Moderately intensifying nutrient-poor permanent grasslands.\n3. Using light grazing instead of heavy grazing.\n4. Increasing the duration of grass leys.\n5. Converting grass leys to grass-legume mixtures or permanent grasslands.\n\nThese practices can help reduce carbon losses and 

##### The first generated answer using window retriever.

In [51]:
pred_response_strs[0]

'Soil carbon sequestration plays a significant role in mitigating greenhouse gas emissions in ruminant production systems by acting as an enhanced sink for greenhouse gases. It can be directly measured by changes in soil organic carbon stocks or indirectly by assessing the net balance of carbon fluxes. Management practices that can enhance carbon sequestration in grasslands include avoiding soil tillage and conversion to arable land, moderately intensifying nutrient-poor permanent grasslands, using light grazing instead of heavy grazing, extending the duration of grass leys, and converting grass leys to grass-legume mixtures or permanent grasslands. These practices help reduce carbon losses and increase the overall carbon sequestration potential of grasslands.'

##### Storing a list of contexts and showing the context of the first generated response using the window retriever.

In [52]:
contexts = []

for i in range(0, max_samples):
    sublist = []
    sublist.append(pred_responses[i].source_nodes[0].node.metadata['window'])
    contexts.append(sublist)

contexts[0]

['Animal (2010), 4:3, pp 334–350 &The Animal Consortium 2009\ndoi:10.1017/S1751731109990784animal\nMitigating the greenhouse gas balance of ruminant production\nsystems through carbon sequestration in grasslands\nJ. F .  Soussana1-, T. Tallec1and V. Blanfort1,2\n1INRA UR0874, UREP Grassland Ecosystem Research, 234, Avenue du Bre ´zet, Clermont-Ferrand, F-63100, France;2CIRAD UR 8, Livestock Systems,\nCampus International de Baillarguet, Cedex 5, Montpellier, F-34398, France\n(Received 6 January 2009; Accepted 12 June 2009; First published online 22 September 2009)\nSoil carbon sequestration (enhanced sinks) is the mechanism responsible for most of the greenhouse gas (GHG) mitigation\npotential in the agriculture sector.  Carbon sequestration in grasslands can be determined directly by measuring changes in soil\norganic carbon (SOC) stocks and indirectly by measuring the net balance of C ﬂuxes.  A literature search shows that grassland\nC sequestration reaches on average 5 630 g C/m2per

##### Storing a list of contexts and showing the context of the first generated response using the base retriever.

In [53]:
base_contexts = []

for i in range(0, max_samples):
    sublist = []
    sublist.append(base_pred_responses[i].source_nodes[0].node.text)
    base_contexts.append(sublist)

base_contexts[0]

['Animal (2010), 4:3, pp 334–350 &The Animal Consortium 2009\ndoi:10.1017/S1751731109990784animal\nMitigating the greenhouse gas balance of ruminant production\nsystems through carbon sequestration in grasslands\nJ. F . Soussana1-, T. Tallec1and V. Blanfort1,2\n1INRA UR0874, UREP Grassland Ecosystem Research, 234, Avenue du Bre ´zet, Clermont-Ferrand, F-63100, France;2CIRAD UR 8, Livestock Systems,\nCampus International de Baillarguet, Cedex 5, Montpellier, F-34398, France\n(Received 6 January 2009; Accepted 12 June 2009; First published online 22 September 2009)\nSoil carbon sequestration (enhanced sinks) is the mechanism responsible for most of the greenhouse gas (GHG) mitigation\npotential in the agriculture sector. Carbon sequestration in grasslands can be determined directly by measuring changes in soil\norganic carbon (SOC) stocks and indirectly by measuring the net balance of C ﬂuxes. A literature search shows that grassland\nC sequestration reaches on average 5 630 g C/m2per ye

## RAGAS Evaluation

In [54]:
from datasets import Dataset 
from ragas import evaluate
from ragas.metrics import (faithfulness, answer_correctness, answer_relevancy, answer_similarity, context_recall)
from ragas.metrics.critique import harmfulness

In [55]:
data_samples = {
    'question': eval_qs[:max_samples],
    'answer': pred_response_strs[:max_samples],
    'ground_truth': ref_response_strs[:max_samples],
    'contexts': contexts[:max_samples]
}

dataset = Dataset.from_dict(data_samples)

score = evaluate(dataset,metrics=[faithfulness, answer_correctness, answer_relevancy, answer_similarity, harmfulness])
score.to_pandas()

Evaluating: 100%|██████████| 50/50 [00:34<00:00,  1.45it/s]


,question,answer,ground_truth,contexts,faithfulness,answer_correctness,answer_relevancy,answer_similarity,harmfulness
0,How does soil carbon sequestration contribute ...,Soil carbon sequestration plays a significant ...,Soil carbon sequestration plays a significant ...,"[Animal (2010), 4:3, pp 334–350 &The Animal Co...",1.000000,0.580267,0.902974,0.987733,0
1,What are the implications of soil carbon seque...,The implications of soil carbon sequestration ...,The implications of soil carbon sequestration ...,"[With nine European sites, direct emissions of...",1.000000,0.747530,0.918362,0.990118,0
2,What are some examples of grassland types and ...,Examples of grassland types and their correspo...,Some examples of grassland types and their cor...,"[Four sites\nbecame CO 2sources in some years,...",0.846154,0.425608,0.991860,0.952431,0
3,"How does the duration of management practices,...","The duration of management practices, such as ...","The duration of management practices, such as ...",[Processes controlling soil organic carbon acc...,1.000000,0.695313,0.925797,0.981253,0
4,How does soil carbon sequestration contribute ...,Soil carbon sequestration is highlighted as th...,Soil carbon sequestration is responsible for m...,[Livestock productionsystems also emit 37% of ...,1.000000,0.997554,0.889519,0.990217,0
5,What is the estimated potential for soil organ...,The estimated potential for soil organic carbo...,The estimated potential for soil organic carbo...,"[Furthermore, it induces 65% of anthro-\npogen...",1.000000,0.999858,0.957192,0.999434,0
6,How did Ogle et al. (2004) find that managemen...,Ogle et al. (2004) found that management pract...,Ogle et al. (2004) found that management pract...,"[Mineral and organic soils (peat, histosoils, ...",1.000000,0.997781,0.950737,0.991125,0
7,What are the advantages and drawbacks of estim...,The advantages of estimating carbon sequestrat...,The advantages of estimating carbon sequestrat...,"[Animal (2010), 4:3, pp 334–350 &The Animal Co...",NaN,0.435338,0.955487,0.991352,0
8,What are the different methods used to assess ...,The different methods used to assess net carbo...,The different methods used to assess net carbo...,[This factor is covered only in part by\nthe 1...,0.250000,0.749088,0.960056,0.996353,0
9,Can you provide examples of specific grassland...,Examples of specific grassland locations and t...,- Alpine extensive pasture and hay meadow at M...,"[Atall sites, the NEE of CO\n2was assessed usi...",0.000000,0.434263,0.961868,0.954491,0


In [56]:
base_data_samples = {
    'question': eval_qs[:max_samples],
    'answer': base_pred_response_strs[:max_samples],
    'ground_truth': ref_response_strs[:max_samples],
    'contexts': base_contexts[:max_samples]
}

base_dataset = Dataset.from_dict(base_data_samples)

score = evaluate(base_dataset,metrics=[faithfulness, answer_correctness, answer_relevancy, answer_similarity, harmfulness])
score.to_pandas()

Evaluating: 100%|██████████| 50/50 [00:26<00:00,  1.88it/s]


,question,answer,ground_truth,contexts,faithfulness,answer_correctness,answer_relevancy,answer_similarity,harmfulness
0,How does soil carbon sequestration contribute ...,Soil carbon sequestration plays a significant ...,Soil carbon sequestration plays a significant ...,"[Animal (2010), 4:3, pp 334–350 &The Animal Co...",1.000000,0.598630,0.964615,0.982756,0
1,What are the implications of soil carbon seque...,The implications of soil carbon sequestration ...,The implications of soil carbon sequestration ...,"[Animal (2010), 4:3, pp 334–350 &The Animal Co...",1.000000,0.792787,0.925829,0.989330,0
2,What are some examples of grassland types and ...,Examples of grassland types mentioned in the d...,Some examples of grassland types and their cor...,[This will require avoidingland use changes th...,1.000000,0.829183,0.952754,0.916733,0
3,"How does the duration of management practices,...","The duration of management practices, such as ...","The duration of management practices, such as ...",[Ogle\net al . (2004) identiﬁed 49 studies\nde...,0.857143,0.423987,0.919160,0.945949,0
4,How does soil carbon sequestration contribute ...,Soil carbon sequestration contributes to the m...,Soil carbon sequestration is responsible for m...,"[Fontaine S, Barot S, Barre P , Bdioui N, Mary...",0.500000,0.486118,0.973280,0.944471,0
5,What is the estimated potential for soil organ...,The estimated potential for soil organic carbo...,The estimated potential for soil organic carbo...,"[Animal (2010), 4:3, pp 334–350 &The Animal Co...",0.000000,0.999769,0.957163,0.999076,0
6,How did Ogle et al. (2004) find that managemen...,Ogle et al. (2004) found that management pract...,Ogle et al. (2004) found that management pract...,[Ogle\net al . (2004) identiﬁed 49 studies\nde...,1.000000,0.999942,0.951705,0.999767,0
7,What are the advantages and drawbacks of estim...,The advantages of estimating carbon sequestrat...,The advantages of estimating carbon sequestrat...,[Ogle\net al . (2004) identiﬁed 49 studies\nde...,1.000000,0.492422,0.954493,0.969690,0
8,What are the different methods used to assess ...,The different methods used to assess net carbo...,The different methods used to assess net carbo...,[Table 1 Literature survey of net C storage (N...,1.000000,0.748839,0.960056,0.995358,0
9,Can you provide examples of specific grassland...,Examples of specific grassland locations and t...,- Alpine extensive pasture and hay meadow at M...,[The uncertainty associated to NCS can be esti...,1.000000,0.517212,0.961868,0.957691,0
